# Tako HRM - Interactive Play

Play against your trained models interactively.

## Available Games

- **TicTacToe** - 3x3 board
- **Othello** - 8x8 board
- **Hex** - 11x11 board
- **Chess** - Standard chess

---

## Verify Setup

**Run `setup.ipynb` first if you haven't already!**

In [ ]:
# Import required libraries
import torch
import yaml
from pathlib import Path
import sys
sys.path.append('.')

from model.hrm import HRM
from training.mcts import MCTS
from games.tictactoe import TicTacToeGame
from games.othello import OthelloGame

# Check device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

---

## TicTacToe

Play 3x3 TicTacToe against your trained model.

In [ ]:
# Verify setup and load libraries
import os
import sys
import torch
import yaml
import numpy as np
from pathlib import Path

if not os.path.exists('scripts/train.py'):
    print("❌ ERROR: Not in tako-v2 directory")
    print("   Run setup.ipynb first!")
    raise FileNotFoundError("Run setup.ipynb first")

sys.path.insert(0, os.getcwd())

from model.hrm import HRM
from training.mcts import MCTS
from games.tictactoe import TicTacToeGame
from games.othello import OthelloGame

# Check device
if torch.cuda.is_available():
    device = 'cuda'
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    device = 'mps'
    print(f"✅ MPS: Apple Silicon GPU")
else:
    device = 'cpu'
    print(f"ℹ️  CPU only")

# Helper function to load model
def load_model(game_name, checkpoint_path=None):
    """Load trained model for a game."""
    config_path = f"config/{game_name}.yaml"
    
    with open(config_path) as f:
        config = yaml.safe_load(f)
    
    model = HRM(**config['model'])
    
    if checkpoint_path is None:
        ckpt_dir = Path(f'checkpoints/{game_name}')
        if ckpt_dir.exists():
            checkpoints = sorted(ckpt_dir.glob('*.pt'), key=lambda p: p.stat().st_mtime)
            if checkpoints:
                checkpoint_path = checkpoints[-1]
            else:
                raise FileNotFoundError(f"No checkpoints found for {game_name}")
        else:
            raise FileNotFoundError(f"Checkpoint directory not found for {game_name}")
    
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()
    
    print(f"✅ Loaded: {Path(checkpoint_path).name}")
    return model, config

print("✅ Setup verified - ready to play!")

In [ ]:
# Play TicTacToe game
game.reset()
print("Starting new game...\n")

while not game.is_terminal():
    # Display board
    print(game.to_string())
    
    if game.current_player == 1:  # Human's turn (X)
        legal = game.legal_moves()
        print(f"Legal moves: {legal}")
        
        move = int(input("Your move (0-8): "))
        while move not in legal:
            print("Illegal move! Try again.")
            move = int(input("Your move (0-8): "))
        
        game.make_move(move)
    else:  # AI's turn (O)
        print("AI thinking...")
        policy = mcts.search(game, move_num=game.move_count)
        # Select best move from policy
        legal = game.legal_moves()
        legal_policy = policy[legal]
        move = legal[np.argmax(legal_policy)]
        print(f"AI plays: {move}\n")
        game.make_move(move)

# Game over
print(game.to_string())
outcome = game.outcome()
if outcome == 1.0:
    print("\n🎉 You win!")
elif outcome == -1.0:
    print("\n🤖 AI wins!")
else:
    print("\n🤝 Draw!")

---

## Othello

Play 8x8 Othello against your trained model.

In [ ]:
# Load Othello model
model, config = load_model('othello')
game = OthelloGame()
mcts = MCTS(model, OthelloGame, config['mcts'], device=device)

print("\nReady to play Othello!")
print("You are Black (B), AI is White (W)")
print("Positions: row (0-7) and column (0-7)\n")

In [ ]:
# Play Othello game
game.reset()
print("Starting new game...\n")

while not game.is_terminal():
    # Display board
    print(game.to_string())
    print(f"Score - Black: {game.score()[0]}, White: {game.score()[1]}")
    
    legal = game.legal_moves()
    
    if not legal:  # Pass if no legal moves
        print(f"Player {game.current_player} passes (no legal moves)\n")
        game.make_move(None)
        continue
    
    if game.current_player == 1:  # Human's turn
        print(f"Legal moves: {legal}")
        
        move_str = input("Your move (row col, e.g. '2 3'): ")
        row, col = map(int, move_str.split())
        move = row * 8 + col
        
        while move not in legal:
            print("Illegal move! Try again.")
            move_str = input("Your move (row col): ")
            row, col = map(int, move_str.split())
            move = row * 8 + col
        
        game.make_move(move)
    else:  # AI's turn
        print("AI thinking...")
        policy = mcts.search(game, move_num=game.move_count)
        # Select best move from policy
        legal_policy = policy[legal]
        move = legal[np.argmax(legal_policy)]
        row, col = move // 8, move % 8
        print(f"AI plays: {row} {col}\n")
        game.make_move(move)

# Game over
print(game.to_string())
final_score = game.score()
print(f"\nFinal Score - Black: {final_score[0]}, White: {final_score[1]}")

outcome = game.outcome()
if outcome == 1.0:
    print("\n🎉 You win!")
elif outcome == -1.0:
    print("\n🤖 AI wins!")
else:
    print("\n🤝 Draw!")

---

## Hex

Play 11x11 Hex against your trained model.

In [ ]:
# Load Hex model
try:
    from games.hex import HexGame
    model, config = load_model('hex')
    game = HexGame()
    mcts = MCTS(model, HexGame, config['mcts'], device=device)
    
    print("\nReady to play Hex!")
    print("You are Red, AI is Blue")
    print("Positions: row (0-10) and column (0-10)\n")
except ImportError:
    print("⚠️  Hex game not yet implemented")
except FileNotFoundError as e:
    print(f"⚠️  {e}")

In [ ]:
# Play Hex game
if 'HexGame' in dir():
    game.reset()
    print("Starting new game...\n")
    
    while not game.is_terminal():
        # Display board
        print(game.to_string())
        
        legal = game.legal_moves()
        
        if game.current_player == 1:  # Human's turn
            print(f"Legal moves: {len(legal)} available")
            
            move_str = input("Your move (row col): ")
            row, col = map(int, move_str.split())
            move = row * 11 + col
            
            while move not in legal:
                print("Illegal move! Try again.")
                move_str = input("Your move (row col): ")
                row, col = map(int, move_str.split())
                move = row * 11 + col
            
            game.make_move(move)
        else:  # AI's turn
            print("AI thinking...")
            policy = mcts.search(game, move_num=game.move_count)
            # Select best move from policy
            legal_policy = policy[legal]
            move = legal[np.argmax(legal_policy)]
            row, col = move // 11, move % 11
            print(f"AI plays: {row} {col}\n")
            game.make_move(move)
    
    # Game over
    print(game.to_string())
    outcome = game.outcome()
    if outcome == 1.0:
        print("\n🎉 You win!")
    elif outcome == -1.0:
        print("\n🤖 AI wins!")
    else:
        print("\n🤝 Draw!")
else:
    print("⚠️  Hex not available")

---

## Chess

Play chess against your trained model using standard algebraic notation.

In [ ]:
# Load Chess model
try:
    from games.chess import ChessGame
    model, config = load_model('chess')
    game = ChessGame()
    mcts = MCTS(model, ChessGame, config['mcts'], device=device)
    
    print("\nReady to play Chess!")
    print("You are White, AI is Black")
    print("Enter moves in algebraic notation (e.g., 'e2e4')\n")
except ImportError:
    print("⚠️  Chess game not yet fully implemented")
except FileNotFoundError as e:
    print(f"⚠️  {e}")

In [ ]:
# Play Chess game
if 'ChessGame' in dir():
    game.reset()
    print("Starting new game...\n")
    
    while not game.is_terminal():
        # Display board
        print(game.to_string())
        
        legal = game.legal_moves()
        
        if game.current_player == 1:  # Human's turn (White)
            print(f"Legal moves: {len(legal)} available")
            
            move_str = input("Your move (e.g. 'e2e4'): ")
            move = game.parse_move(move_str)
            
            while move not in legal:
                print("Illegal move! Try again.")
                move_str = input("Your move: ")
                move = game.parse_move(move_str)
            
            game.make_move(move)
        else:  # AI's turn (Black)
            print("AI thinking...")
            policy = mcts.search(game, move_num=game.move_count)
            # Select best move from policy
            legal_policy = policy[legal]
            move = legal[np.argmax(legal_policy)]
            move_str = game.move_to_string(move)
            print(f"AI plays: {move_str}\n")
            game.make_move(move)
    
    # Game over
    print(game.to_string())
    outcome = game.outcome()
    if outcome == 1.0:
        print("\n🎉 You win!")
    elif outcome == -1.0:
        print("\n🤖 AI wins!")
    else:
        print("\n🤝 Draw!")
else:
    print("⚠️  Chess not available")

---

## Watch AI vs AI

Watch two versions of your model play against each other.

In [ ]:
# AI vs AI game
GAME = "tictactoe"  # Change to othello, hex, chess

# Load model
if GAME == "tictactoe":
    game_class = TicTacToeGame
elif GAME == "othello":
    game_class = OthelloGame
else:
    print(f"⚠️  {GAME} not available for AI vs AI yet")
    game_class = None

if game_class:
    model, config = load_model(GAME)
    game = game_class()
    mcts = MCTS(model, game_class, config['mcts'], device=device)
    
    game.reset()
    print(f"Starting AI vs AI {GAME} game...\n")
    
    while not game.is_terminal():
        print(game.to_string())
        
        policy = mcts.search(game, move_num=game.move_count)
        # Select best move from policy
        legal = game.legal_moves()
        legal_policy = policy[legal]
        move = legal[np.argmax(legal_policy)]
        print(f"Player {game.current_player} plays: {move}\n")
        game.make_move(move)
        
        import time
        time.sleep(0.5)  # Pause for visualization
    
    print(game.to_string())
    outcome = game.outcome()
    if outcome == 1.0:
        print("\nPlayer 1 wins!")
    elif outcome == -1.0:
        print("\nPlayer 2 wins!")
    else:
        print("\nDraw!")